In [1]:
cd /home/c01muya/vae-disentanglement/disentanglement_lib_pl

/home/c01muya/vae-disentanglement/disentanglement_lib_pl


In [2]:
import numpy as np
import pandas as pd
import seaborn, time

from pomegranate import BayesianNetwork
from common.data_loader import DSpritesDataset, ThreeShapesDataset, ContinumDataset
from common.known_datasets import CorrelatedDSpritesDataset


In [3]:
seaborn.set_style('whitegrid')

In [4]:
def sanitize_celeb_attr_data(unsanitize_attrs_file):
    
    attr_list = []
    
    unsanitize_attrs = open(unsanitize_attrs_file,'r').readlines()
    
    for a in unsanitize_attrs:
        attr_line = a.strip('\n').split(' ')
        attrs_sanitized = [attr for attr in attr_line if attr != '']
        attr_list.append(attrs_sanitized)
        
    with open('/home/c01muya/vae-disentanglement/datasets/celeba/list_attr_celeba_sanitized.txt','w') as list_attr_celeb_sen:
        for attr_line in attr_list:
            list_attr_celeb_sen.write(
                ",".join(attr_line) + "\n"
            )

In [5]:
#sanitize_celeb_attr_data("/home/c01muya/vae-disentanglement/datasets/celeba/list_attr_celeba.txt")

In [40]:
celeb_attrs = pd.read_csv("/home/c01muya/vae-disentanglement/datasets/celeba/list_attr_celeba_sanitized.txt", 
                          sep=",", encoding='ascii')
celeb_attrs.drop('Image_ID', axis='columns', inplace=True)
celeb_attrs.head()

,5_o_Clock_Shadow,Arched_Eyebrows,Attractive,Bags_Under_Eyes,Bald,Bangs,Big_Lips,Big_Nose,Black_Hair,Blond_Hair,...,Sideburns,Smiling,Straight_Hair,Wavy_Hair,Wearing_Earrings,Wearing_Hat,Wearing_Lipstick,Wearing_Necklace,Wearing_Necktie,Young
0,-1,1,1,-1,-1,-1,-1,-1,-1,-1,...,-1,1,1,-1,1,-1,1,-1,-1,1
1,-1,-1,-1,1,-1,-1,-1,1,-1,-1,...,-1,1,-1,-1,-1,-1,-1,-1,-1,1
2,-1,-1,-1,-1,-1,-1,1,-1,-1,-1,...,-1,-1,-1,1,-1,-1,-1,-1,-1,1
3,-1,-1,1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,1,-1,1,-1,1,1,-1,1
4,-1,1,1,-1,-1,-1,1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,1,-1,-1,1


In [41]:

hair_color_conditions = [
    celeb_attrs['Black_Hair'] == 1,
    celeb_attrs['Brown_Hair'] == 1,
    celeb_attrs['Blond_Hair'] == 1,
    celeb_attrs['Gray_Hair']  == 1,
    (celeb_attrs['Black_Hair'] != 1) & (celeb_attrs['Brown_Hair'] != 1) & (celeb_attrs['Blond_Hair'] != 1) & (celeb_attrs['Gray_Hair']  != 1)
]

hair_color_values = ['Black', 'Brown', 'Blond', 'Gray', 'Unknown']
celeb_attrs['Hair_Color'] = np.select(hair_color_conditions, hair_color_values)
#celeb_attrs[ ['Hair_Color','Black_Hair', 'Brown_Hair', 'Blond_Hair', 'Gray_Hair'] ]
celeb_attrs.drop(['5_o_Clock_Shadow', 'Black_Hair','Brown_Hair','Blond_Hair','Gray_Hair'], axis='columns', inplace=True)

In [42]:
hair_type_conditions = [
    celeb_attrs['Bald'] == 1,
    celeb_attrs['Bangs'] == 1,
    celeb_attrs['Straight_Hair'] == 1,
    celeb_attrs['Wavy_Hair']  == 1,
    (celeb_attrs['Bald'] != 1) & (celeb_attrs['Bangs'] != 1) & (celeb_attrs['Straight_Hair'] != 1) & (celeb_attrs['Wavy_Hair'] != 1)
]

hair_type_values = ['Bald', 'Bangs', 'Straight', 'Wavy','Unknown']

celeb_attrs['Hair_Type'] = np.select(hair_type_conditions, hair_type_values)
#celeb_attrs[ ['Hair_Type','Bald', 'Bangs', 'Straight_Hair', 'Wavy_Hair'] ]
celeb_attrs.drop(['Bald', 'Bangs', 'Straight_Hair', 'Wavy_Hair'], axis='columns', inplace=True)

In [44]:
celeb_attrs.head(10)

,Arched_Eyebrows,Attractive,Bags_Under_Eyes,Big_Lips,Big_Nose,Blurry,Bushy_Eyebrows,Chubby,Double_Chin,Eyeglasses,...,Sideburns,Smiling,Wearing_Earrings,Wearing_Hat,Wearing_Lipstick,Wearing_Necklace,Wearing_Necktie,Young,Hair_Color,Hair_Type
0,1,1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,1,1,-1,1,-1,-1,1,Brown,Straight
1,-1,-1,1,-1,1,-1,-1,-1,-1,-1,...,-1,1,-1,-1,-1,-1,-1,1,Brown,Unknown
2,-1,-1,-1,1,-1,1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,1,Unknown,Wavy
3,-1,1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,1,-1,1,1,-1,1,Unknown,Straight
4,1,1,-1,1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,1,-1,-1,1,Unknown,Unknown
5,1,1,-1,1,-1,-1,-1,-1,-1,-1,...,-1,-1,1,-1,1,-1,-1,1,Brown,Wavy
6,-1,1,1,1,1,-1,1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,1,Black,Straight
7,1,-1,1,1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,1,Black,Unknown
8,1,1,-1,1,-1,-1,-1,-1,-1,-1,...,-1,1,1,-1,1,-1,-1,1,Unknown,Bangs
9,-1,1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,1,-1,-1,1,Unknown,Wavy


In [45]:
celeb_attrs_mat = celeb_attrs.to_numpy()
print(celeb_attrs_mat.shape)

(202599, 33)


In [46]:
celeb_attrs_mat_subset = celeb_attrs_mat[:10000,:]
print(celeb_attrs_mat_subset.shape)

(10000, 33)


In [47]:
model = BayesianNetwork.from_samples(celeb_attrs_mat_subset, 
                                     state_names=celeb_attrs.columns.to_list(), 
                                     algorithm='chow-liu')
print(model.structure)

((), (27,), (4,), (0,), (13,), (1,), (31,), (4,), (7,), (1,), (17,), (27,), (27,), (27,), (24,), (17,), (14,), (13,), (12,), (12,), (11,), (31,), (11,), (17,), (12,), (13,), (31,), (0,), (13,), (13,), (1,), (13,), (27,))


In [49]:
#%matplotlib inline

model.plot("../reports/learned-dag-3.pdf")

In [ ]:
# 1. The labels in celeba don't capture the variation within each attribute value. Not all blond hair at the same for example. 
# So we shouldn't use just a categorical to represent that information.


In [5]:
import os

os.environ['DISENTANGLEMENT_LIB_DATA'] = '../datasets/'
print(os.environ.get('DISENTANGLEMENT_LIB_DATA','.'))
corr_dsprites = CorrelatedDSpritesDataset(correlation_strength=0.2, seed=123, split='train')

../datasets/
Initialize [CorrelatedDSpritesDataset] with 737280 examples. Shape (737280, 64, 64).


In [18]:
factors, Xs = corr_dsprites.dataset.sample(10000, random_state=corr_dsprites.random_state)

In [19]:
factors.shape,Xs.shape

((10000, 5), (10000, 64, 64, 1))

In [22]:

dsprites_model = BayesianNetwork.from_samples(factors, 
                                     state_names=['shape','scale','orientation','posX', 'posY'], 
                                     algorithm='exact')
print(dsprites_model.structure)

((), (), (3,), (), ())


In [21]:
dsprites_model.plot("../reports/dsprites-dag.pdf")

ValueError: must have matplotlib and pygraphviz installed for visualization